In [0]:
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
import tensorflow as tf
import numpy as np

In [0]:
class MobileNetBlock(layers.Layer):
  def __init__(self, num_filters, strides):
    super(MobileNetBlock, self).__init__()
    self.depthwise_layer = layers.DepthwiseConv2D(kernel_size=3, strides=strides, padding='same')
    self.batchnorm_layer_1 = layers.BatchNormalization()
    self.conv_layer = layers.Conv2D(filters=num_filters, kernel_size=1, strides=1, padding='same')
    self.batchnorm_layer_2 = layers.BatchNormalization()
  def call(self, tensor):
    x = self.depthwise_layer(tensor)
    x = self.batchnorm_layer_1(x)
    x = layers.ReLU()(x)
    x = self.conv_layer(x)
    x = self.batchnorm_layer_2(x)
    x = layers.ReLU()(x)
    return x

In [0]:
input_layer = layers.Input((224, 224, 3))
x = layers.Conv2D(filters=32, kernel_size=3, strides=2, padding='same')(input_layer)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = MobileNetBlock(64, 1)(x)
x = MobileNetBlock(128, 2)(x)
x = MobileNetBlock(128, 1)(x)
x = MobileNetBlock(256, 2)(x)
x = MobileNetBlock(256, 1)(x)
x = MobileNetBlock(512, 2)(x)
for _ in range(5):
  x = MobileNetBlock(512, 1)(x)
x = MobileNetBlock(1024, 2)(x)
x = MobileNetBlock(1024, 1)(x)
x = layers.AveragePooling2D(pool_size=7, strides=1)(x)
output_layer = layers.Dense(10, activation=tf.nn.softmax)(x)
model = Model(input_layer, output_layer)